# 06 Offices_Electric_Active Energy (kWh)

In [2]:
cd ..

/Users/johanbarreiro/Documents/GitHub/vl_optimizer


## Definition of the constrain equation

**Offices_Electric_Active Energy = A⋅lightbulbs_active + B⋅active_wall_plugs + C⋅active_computers + D⋅active_printers + E⋅active_coffee_machines**

- **Offices_Electric_Active Energy** = Energy consumption of the Office building in a given period.

- **lightbulbs_active** = Represents the number of active lightbulbs during the specified period - Given.

- **active_wall_plugs** = Refers to the number of wall plugs currently in use or drawing power - Given.

- **active_computers** = Indicates the number of computers currently in use - Given.

- **active_printers** = The number of printers in active use during the measurement period - Controllable.

- **active_coffee_machines** = Represents the number of coffee machines currently in use - Controllable.

- **A,B,C,D,E** = Coefficients for each factor.

## Other factors to include in the constrain equation

1. **lightbulbs_active**: The activity level of lightbulbs should scale with the lighting energy consumption, which could be a significant fraction of the total office energy use.
2. **active_wall_plugs**: The number of active wall plugs should correlate with the energy consumption from plugged devices.
3. **active_computers**: Computer use should scale with the proportion of energy dedicated to IT and office equipment.
4. **active_printers**: Printer usage should align with the specific energy consumption of these devices.
5. **active_coffee_machines**: The operation of coffee machines should scale with their share of energy use.

We will use the percentile distribution of energy consumption to dynamically adjust the maximum limits for variables, which are more directly influenced by the level of activity in the office.

Let’s start by analyzing the total energy consumption to inform these decisions.

Here are some key statistics from the total energy consumption data:

## Get cofficients

In [9]:
import pandas as pd
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from joblib import dump

# Define the target
target = office_target

# Define the features
features = df_simulated.columns

# Split the data into features (X) and target (y)
X = df_simulated
y = target

# Initialize the linear regression model
model = LinearRegression()

# Perform k-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
cv_results_mse = cross_val_score(model, X, y, cv=kfold, scoring='neg_mean_squared_error')
cv_results_r2 = cross_val_score(model, X, y, cv=kfold, scoring='r2')

# Fit the model on the entire dataset
model.fit(X, y)

# Get the intercept and coefficients
intercept = model.intercept_
coefficients = model.coef_

# Save the trained model to a file
model_params = {
    'intercept': intercept,
    'coefficients': coefficients,
    'features': features
}
model_filename = 'equation_modeling/models/office_consumption_model.joblib'
dump(model_params, model_filename)

# Calculate and print performance metrics
y_pred = model.predict(X)
mse = mean_squared_error(y, y_pred)
r2 = r2_score(y, y_pred)

print(f'Mean Squared Error (Cross-Validation): {-cv_results_mse.mean()}')
print(f'Standard Deviation (Cross-Validation): {cv_results_mse.std()}')
print(f'R^2 Score (Cross-Validation): {cv_results_r2.mean()}')
print(f'Overall R^2 Score: {r2}')

# Optional: Print the coefficients of the model
coefficients_df = pd.DataFrame(coefficients, features, columns=['Coefficient'])
print(f'Intercept: {intercept}')
print(coefficients_df)


Mean Squared Error (Cross-Validation): 1.277939958175431
Standard Deviation (Cross-Validation): 0.046029413806609486
R^2 Score (Cross-Validation): 0.902095097226141
Overall R^2 Score: 0.9023330893490779
Intercept: 0.26039035147439904
                        Coefficient
lightbulbs_active          0.608375
active_wall_plugs          0.480035
active_computers           0.974240
active_printers            3.604773
active_coffee_machines     9.465781
